In [63]:
# Handle to the workspace
from azure.ai.ml import MLClient

# Authentication package
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential, AzureCliCredential

try:
    credential = AzureCliCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

AzureCliCredential.get_token failed: ERROR: AADSTS70043: The refresh token has expired or is invalid due to sign-in frequency checks by conditional access. The token was issued on 2025-04-01T06:22:55.4629892Z and the maximum allowed lifetime for this request is 14400. Trace ID: 45106e2e-d82f-4110-82b1-0205d53c1700 Correlation ID: 53c5813f-6bd2-4ee0-be52-146ae5291245 Timestamp: 2025-04-03 06:57:52Z
Interactive authentication is needed. Please run:
az login --scope https://management.azure.com/.default



In [64]:
ml_client = MLClient.from_config(credential)

Found the config file in: /home/daniel/repos/aml_demo/config.json


Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


In [65]:
import os
os.makedirs("../components/registeryolomodel", exist_ok=True)

In [66]:
%%writefile ../components/registeryolomodel/register_yolo_model.py

import os
import argparse
import ultralytics

def main():
    parser = argparse.ArgumentParser(description="Register a fresh ultralytics model")
    parser.add_argument(
        "--model_type",
        type=str,
        default="yolo11n.pt",
        help="The model to register. Default is yolo11n.pt",
    )
    # parser.add_argument("--model_name", type=str, default = f"{parser.parse_args().model}_test", help="The name of the model to register")


    model = ultralytics.YOLO(parser.parse_args().model_type)
    # model_name = parser.parse_args().model_name
    print(f"Model {parser.parse_args().model_type} loaded successfully")
    # save the model to a local directory
    os.mkdir("./model")
    model.save("./model/testmodel.pt")
    print("Model saved successfully")
    print(os.listdir("./model"))
if __name__ == "__main__":
    main()



Overwriting ../components/registeryolomodel/register_yolo_model.py


In [67]:
from azure.ai.ml import command, Input, Output
from azure.ai.ml.constants import AssetTypes

inputs = {"model_type": Input(type="string")}
outputs = {"model": Output(type=AssetTypes.CUSTOM_MODEL, description="The name of the model to register")}

register_yolo_model = command(
    name = "register_yolo_model",
    code="../components/registeryolomodel",
    inputs=inputs,
    outputs=outputs,
    command="python register_yolo_model.py --model_type ${{inputs.model_type}}",
    environment="yolofromdocker:1",
    compute="defaultcompute" )

job = register_yolo_model()

In [60]:
from azure.ai.ml import dsl

In [68]:
@dsl.pipeline(description="Yolo Registration Pipeline", name="yolo_registration_pipeline")
def yolo_registration_pipeline(
    model_type: str = "yolo11n.pt",
):
    register_yolo_model(
        model_type=model_type,
    )

In [62]:
pipeline = yolo_registration_pipeline(
    model_type="yolo11n.pt",
)
ml_client.jobs.create_or_update(pipeline, experiment_name="yolo_registration_experiment")

Uploading registeryolomodel (0.0 MBs): 100%|██████████| 852/852 [00:00<00:00, 22837.52it/s]


pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.CustomModelJobOutput'> and will be ignored


Experiment,Name,Type,Status,Details Page
yolo_registration_experiment,purple_fox_gb2ss0z0pp,pipeline,NotStarted,Link to Azure Machine Learning studio
